1 - Setup

In [ ]:
# mount + imports
from google.colab import drive
drive.mount('/content/drive')
DATA_ROOT = '/content/drive/My Drive/ChatBot'

import json, string, random
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# NLTK data needed by word_tokenize + lemmatizer
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("wordnet")

# optional but useful for lemmatizer language data
nltk.download("omw-1.4")

# load intents
with open(f'{DATA_ROOT}/intents.json') as f:
  data = json.load(f)

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


2 - Data Prep (vocab + training set)

In [ ]:
# build words/classes from intents
words = [] #for bag-of-words (BoW) model/ vocab for patterns
classes = [] #for BoW model/ vocab for tags
data_X = [] #for storing patterns
data_Y = [] #for storing tag corresponding to pattern in data_X

# iterating for all intents
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern) # tokenizing patterns
    words.extend(tokens) # appending tokens to words
    data_X.append(pattern) # appending patterns to data_X
    data_Y.append(intent["tag"]) # appending associated tag to patterns

  # adding tag to classes if not already there
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

# lemmatizing words in vocab and converting to lowercase
# if words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower())
  for word in words if word not in string.punctuation]

# sorting vocab and classes in alphabetical order, taking # set to ensure no duplicates
words = sorted(set(words))
classes = sorted(set(classes))

# converting text to numbers, building training set via BoW
training = []
out_empty = [0] * len(classes)

# creating BoW model

for idx, doc in enumerate(data_X):
  # 1) tokenize each training sentence (pattern)
  #    e.g., "How are you?" -> ["How", "are", "you", "?"]
  tokens = nltk.word_tokenize(doc)

  # 2) normalize tokens (lowercase + lemmatize) and remove punctuation
  #    ensures "running" -> "run", "Hello" -> "hello"
  tokens = [lemmatizer.lemmatize(w.lower())
    for w in tokens if w not in string.punctuation]

  # 3) build Bo@ vector for this sentence
  #    go through the *entire vocabulary* (words)
  #    put 1 if vocab word appears in this sentence's tokens, else 0
  bow = [1 if w in tokens else 0 for w in words]

  # 4) build the one-hot label vector for sentence's intent (class)
  label = list(out_empty)
  label[classes.index(data_Y[idx])] = 1

  # 5) add this training example (features + label) to the dataset
  training.append([bow, label])

# shuffle data and convert to array
random.shuffle(training)
training = np.array(training, dtype=object)

# split features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))


3A - Train & Save Model

In [ ]:
# Neural Network Model definition
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())

# Train
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

# persist model + vocab so we can skip retraining later
MODEL_PATH = f'{DATA_ROOT}/model.keras' # .keras format preferred
VOCAB_PATH = f'{DATA_ROOT}/vocab.json'

model.save(MODEL_PATH)
with open(VOCAB_PATH, 'w') as f:
  json.dump({'words': words, 'classes': classes}, f)

print("Saved:", MODEL_PATH, "and", VOCAB_PATH)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 18)             │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,922 (77.82 KB)

 Trainable params: 19,922 (77.82 KB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.0434 - loss: 2.9315
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1425 - loss: 2.7916
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.2198 - loss: 2.7087 
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2971 - loss: 2.5101
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2980 - loss: 2.4023
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.3501 - loss: 2.2470 
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3284 - loss: 2.1366 
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5169 - loss: 1.8270 
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5169 - loss: 1.7051
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5839 - loss: 1.3968 
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.4405 - loss: 1.5424 
Epoch 12/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 

3B - Reload Model (use instead of training)

In [ ]:
# Use this INSTEAD of Cell 3A above in a fresh session
from tensorflow.keras.models import load_model
MODEL_PATH = f'{DATA_ROOT}/model.keras'
model = load_model(MODEL_PATH)
print("Model loaded.")



Model loaded.


3B.1 - Reload Vocab

In [ ]:
VOCAB_PATH = f'{DATA_ROOT}/vocab.json'
with open(VOCAB_PATH) as f:
    d = json.load(f)
words, classes = d['words'], d['classes']
print("Vocab loaded. | words:", len(words), "| classes:", len(classes))

Vocab loaded. | words: 81 | classes: 18


3B.1 - Sanity Check

In [ ]:
assert model.output_shape[-1] == len(classes), "Model/classes size mismatch."
print("OK: model outputs =", model.output_shape[-1], "| classes =", len(classes))

OK: model outputs = 18 | classes = 18


Cell 4 - Helper Functions

In [ ]:
# pre-processing user input
def clean_text(text):
  # tokenize -> lowercase -> lemmatize -> drop punctuation
  tokens = nltk.word_tokenize(text)
  return [lemmatizer.lemmatize(w.lower()) for w in tokens if w not in string.punctuation]


def bag_of_words(text, vocab):
  # vector length == len(vocab); 1 if token present else 0
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  return np.array([1 if w in tokens else 0 for w in vocab])

def pred_class(text, vocab, labels):
  """
  Map a raw user message to one or more intent labels, ranked by probability.

  Inputs:
    text   : raw user message (string)
    vocab  : list of vocabulary terms used to build BoW vectors
    labels : list of class names (intent tags), aligned with model outputs

  Outputs:
    A list of intent labels whose probability > threshold, sorted desc by prob.
    Empty list if nothing clears the threshold (lets get_response handle fallback).
  """

  # 1) convert raw text to same BoW representation used in training
  bow = bag_of_words(text, vocab) #shape: (len(vocab),)

  # 2) model inference -> probability distribution over classes
  #    verbose = 0 keeps Colab output clean
  probs = model.predict(np.array([bow]), verbose=0)[0] #shape: (num_classes,)

  # 3) keep only confident predctions
  thresh = 0.5              # tune as needed (0.3..0.7 typical)
  #    collect (class_index, probability) for items above threshold
  candidates = [[i,p] for i ,p in enumerate(probs) if p > thresh]

  # 4) rank by probability, highest first
  candidates.sort(key=lambda x: x[1], reverse=True)

  # 5) return labels for ranked candidates
  return [labels[i] for i, _ in candidates]

def get_response(intents_list, intents_json):
  """
  Select a response for the top predicted intent.
  Falls back to a default message if no intent passes threshold
  or the tag is missing in the intents file.
  """

  # 1) no intents predicted above threshold -> return fallback
  if not intents_list:
    return "Sorry! I don't understand."

  # 2) use the highest probability intent (index 0 from pred_class)
  top_tag = intents_list[0]

  # 3) find the matching intent block in the loaded intents JSON
  for intent in intents_json.get("intents", []):
      if intent.get("tag") == top_tag:
        # 4) pick a random canned response for variability
        responses = intent.get("responses", [])
        if responses:
          return random.choice(responses)
        # tag found but no responses defined -> safe fallback
        return "Sorry! I don't understand."
  # 5) tag not found in JSON (data drift or mismatch) -> fallback
  return "Sorry! I don't understand."



4.1 - Skills Helper

In [ ]:
# import skill helpers (rule-based)
import re, ast, operator as op

# Safe math eval: allow + - * / // % ** and parentheses only
_allowed_ops = {
    ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul, ast.Div: op.truediv,
    ast.FloorDiv: op.floordiv, ast.Mod: op.mod, ast.Pow: op.pow, ast.USub: op.neg
}
def _eval_ast(node):
    if isinstance(node, ast.Num):  # 3.8: ast.Constant in newer Python; Colab Py3.12 still supports Num for ints
        return node.n
    if isinstance(node, ast.UnaryOp) and type(node.op) in _allowed_ops:
        return _allowed_ops[type(node.op)](_eval_ast(node.operand))
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_ops:
        return _allowed_ops[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    raise ValueError("disallowed")

def try_math(message: str):
    expr = message.strip().replace(" ", "")
    # accept forms like 1+2, 10-7, 3*4, 12/3, -5+2, (2+3)*4, 2**3
    if re.fullmatch(r"[0-9\(\)\+\-\*/% ]+|\-?[0-9]+(\.\d+)?([+\-*/%]\-?[0-9]+(\.\d+)?)*", message.strip()):
        try:
            val = _eval_ast(ast.parse(expr, mode="eval").body)
            return f"{val}"
        except Exception:
            return None
    return None

CAPITALS = {
    "norway": "Oslo",
    "sweden": "Stockholm",
    "denmark": "Copenhagen",
    "finland": "Helsinki",
    "france": "Paris",
    "spain": "Madrid",
    "germany": "Berlin",
    "italy": "Rome",
    "japan": "Tokyo",
    "united states": "Washington, D.C.",
    "usa": "Washington, D.C.",
    "canada": "Ottawa",
    "mexico": "Mexico City",
    "india": "New Delhi",
    "china": "Beijing",
    "australia": "Canberra",
    "uk": "London",
    "united kingdom": "London"
}
_capital_pat = re.compile(r"(?:what\s+is\s+)?the\s+capital\s+of\s+(.+)\??", re.I)

def try_capital(message: str):
    m = _capital_pat.search(message)
    if not m:
        return None
    country = m.group(1).strip().lower()
    # normalize some punctuation and quotes
    country = re.sub(r"[^\w\s\.]", "", country)
    # simple lookup
    cap = CAPITALS.get(country)
    return cap if cap else "I don’t have that country in my local list."


Cell 5 - Chat Loop

In [ ]:
# interacting with chatbot
# keep this loop in its own cell in Colab.
# it reads user input and prints chatbot replies until you type "0".

print("Press 0 if you don't want to chat with the ChatBot.")

while True:
  try:
    # prompt user for input (single line)
    message = input("You: ").strip()
  except EOFError:
    # happens if input is interrupted or cell stopped
    print("Chat ended unexpectedly.")
    break

  # compare to the string "0", not the integer 0
  if message == "0":
    print("Chat ended.")
    break

  # try rule-based skills first
  ans = try_math(message)
  if ans is not None:
      print("Bot:", ans)
      continue

  ans = try_capital(message)
  if ans is not None:
      print("Bot:", ans)
      continue

  # otherwise, fall back to intent model
  # predict intent(s) from the user message using the trained model
  intents = pred_class(message, words, classes)

  # map top intent to a response (or fallback if none meet threshold)
  result = get_response(intents, data)

  # show bot reply
  print("Bot:", result)

Press 0 if you don't want to chat with the ChatBot.
You: hi
Bot: Hey.
You: hello
Bot: Hi.
You: wha'ts up
Bot: Waiting for your next question.
You: what do you do
Bot: I can do many things. For example, ask me for the capital, currency, and area of a country; a random number; or to calculate a math problem.
You: what's the capital of america
Bot: I don’t have that country in my local list.
You: what about the US?
Bot: I'm a simple chatbot trained on this intents file.
You: what's the capitcal of the US>
Bot: I work to serve you as well as possible.
You: what's the capital of the US?
Bot: I don’t have that country in my local list.
You: what's the capital of Norway?
Bot: Oslo
You: what's the currency of Norway?
Bot: I work to serve you as well as possible.
You: what is 1+2?
Bot: My job is to assist you.
You: 1+2


/tmp/ipython-input-3798308043.py:10: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # 3.8: ast.Constant in newer Python; Colab Py3.12 still supports Num for ints
/tmp/ipython-input-3798308043.py:11: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n


Bot: 3
You: 3*8
Bot: 24
You: what else do you do?
Bot: I can do many things. For example, ask me for the capital, currency, and area of a country; a random number; or to calculate a math problem.
You: i'm feeling good
Bot: Good to hear. Anything you need?
You: i'm feeling bad
Bot: Noted. Do you want to talk about it?
You: yes 
Bot: Sorry! I don't understand.
You: I want to talk about how i feel bad
Bot: Understood. Do you want resources or to change the topic?
You: I want resources
Bot: Good to hear. Anything you need?
You: 0
Chat ended.
